In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import csv
data={}
with open('/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]

In [5]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [6]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [7]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [13]:
from shutil import copyfile
import os

split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size * len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]

    # Copy train images
    for image in train_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/training', emotion, image)
        copyfile(source, dest)

    # Copy test images
    for image in test_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/testing', emotion, image)
        copyfile(source, dest)



In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense, Flatten

In [25]:
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(8,activation='softmax')


])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 49, 49, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 10, 10, 64)       

In [26]:
training_dir='/content/master_data/training'
testing_dir='/content/master_data/testing'
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
                                                  training_dir,
                                                  target_size=(100,100),
                                                  class_mode='categorical',
                                                  batch_size=128
                                                  )
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
                                                  testing_dir,
                                                  target_size=(100,100),
                                                  class_mode='categorical',
                                                  batch_size=128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [27]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [31]:
model.fit_generator(train_generator,
                    epochs=5,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

<ipython-input-31-05e47a449e80>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,
Exception ignored in: <function _xla_gc_callback at 0x7c9f236cd630>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Epoch 1/20
86/86 [==============================] - 74s 863ms/step - loss: 0.1611 - acc: 0.9389 - val_loss: 2.4889 - val_acc: 0.6838
Epoch 2/20
86/86 [==============================] - 79s 919ms/step - loss: 0.1267 - acc: 0.9536 - val_loss: 2.8570 - val_acc: 0.6689
Epoch 3/20
86/86 [==============================] - 74s 864ms/step - loss: 0.1009 - acc: 0.9634 - val_loss: 3.3170 - val_acc: 0.6787
